In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install https://github.com/bitranox/wrapt-timeout-decorator/archive/master.zip
    

In [1]:
import time
from wrapt_timeout_decorator import *

@timeout(5)
def mytest(message):
    print(message)
    for i in range(1,10):
        time.sleep(1)
        print('{} seconds have passed'.format(i))

if __name__ == '__main__':
    mytest('starting')

starting
1 seconds have passed
2 seconds have passed
3 seconds have passed
4 seconds have passed


TimeoutError: Function mytest timed out after 5 seconds

In [1]:
import time
from wrapt_timeout_decorator import *

@timeout(5, use_signals=False)
def mytest(message):
    print(message)
    for i in range(1,10):
        time.sleep(1)
        print('{} seconds have passed'.format(i))

if __name__ == '__main__':
    mytest('starting')

starting
1 seconds have passed
2 seconds have passed
3 seconds have passed
4 seconds have passed


TimeoutError: Function mytest timed out after 5 seconds